In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True, device='cpu'):
        super(FocalLoss, self).__init__()
        """
        gamma(int) : focusing parameter.
        alpha(list) : alpha-balanced term.
        size_average(bool) : whether to apply reduction to the output.
        """
        self.gamma = gamma
        self.alpha = alpha
        self.size_average = size_average
        self.device = device

    def forward(self, input, target):
        # input : N * C (btach_size, num_class)
        # target : N (batch_size)

        CE = F.cross_entropy(input, target, reduction='none')  # -log(pt)
        pt = torch.exp(-CE)  # pt
        loss = (1 - pt) ** self.gamma * CE  # -(1-pt)log(pt)

        if self.alpha is not None:
            alpha = torch.tensor(self.alpha, dtype=torch.float).to(self.device)
            # in case that a minority class is not selected when mini-batch sampling
            if len(self.alpha) != len(torch.unique(target)):
                temp = torch.zeros(len(self.alpha)).to(self.device)
                temp[torch.unique(target)] = alpha.index_select(0, torch.unique(target))
                alpha_t = temp.gather(0, target)
                loss = alpha_t * loss
            else:
                alpha_t = alpha.gather(0, target)
                loss = alpha_t * loss

        if self.size_average:
            loss = torch.mean(loss)

        return loss

In [2]:
input = torch.randn(10, 3)
target = torch.tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1], dtype = torch.int64)

print(input)
print(target)

tensor([[ 0.5150,  2.1537, -0.0139],
        [-0.9463, -0.7248, -0.7565],
        [ 0.6822, -1.5298, -2.2776],
        [-0.1918, -0.6967, -2.9719],
        [-1.5415,  0.9901, -0.9495],
        [-2.1735,  1.7428,  0.5037],
        [-1.9944, -0.8472, -0.7253],
        [ 0.3618, -1.0586, -0.4950],
        [ 1.7761,  1.1924,  1.1938],
        [-1.4052,  0.8607, -0.4928]])
tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1])


In [3]:
F.cross_entropy(input, target, reduction = 'mean')

tensor(1.5195)

In [4]:
FL = FocalLoss(gamma = 0, size_average = True)
FL(input, target)

tensor(1.5195)

In [5]:
FL = FocalLoss(gamma = 5, size_average = True)
FL(input, target)

tensor(0.8151)

In [6]:
FL = FocalLoss(gamma = 5, alpha = [0.1, 0.7, 0.2], size_average = True)
FL(input, target)

tensor(0.1557)

In [5]:
import numpy as np
arr = np.random.randn(99, 15, 17)
np.concatenate([arr, np.nan])

array([[[ 2.18573670e-01, -1.09015152e+00, -2.69082212e+00, ...,
         -1.49333926e+00,  1.12770970e+00, -1.98849061e+00],
        [ 1.71868894e+00,  1.01534847e+00,  1.62848327e+00, ...,
          5.12436603e-02, -2.01925104e+00,  8.94938775e-01],
        [-7.76215994e-01, -5.26727243e-01,  2.38813286e+00, ...,
         -1.18040853e+00, -5.18561601e-01, -6.07886523e-01],
        ...,
        [ 1.92206385e-01, -1.81492593e+00, -1.20385230e+00, ...,
          9.19349556e-01, -2.98661244e-01, -1.95589469e+00],
        [ 1.65530608e+00,  1.17063079e+00, -2.39808291e+00, ...,
          1.34965446e+00,  1.25916186e+00, -6.06356318e-02],
        [-2.74208401e+00,  4.73398204e-02, -1.97775115e+00, ...,
          7.38647522e-01, -6.01786625e-01,  1.32696211e+00]],

       [[ 4.71637191e-01,  2.27699562e-01,  9.46043829e-01, ...,
          6.91935757e-01,  1.98037802e+00, -9.97883123e-01],
        [ 3.08345832e-01,  1.35438811e+00,  1.54770231e-01, ...,
         -8.72665473e-01,  5.12519885e

In [4]:
nan_idx = np.where(np.isnan(arr))[0]
print(nan_idx)
arr = np.delete(arr, nan_idx)
arr

[]


array([ 1.,  2.,  3., inf,  4.,  5.])

In [53]:
inf_idx = np.where(np.isinf(arr))[0]
print(inf_idx)
arr = np.delete(arr, inf_idx)
arr

[3]


array([1., 2., 3., 4., 5.])

In [20]:
arr = torch.tensor(arr)
arr

tensor([1., 2., 3., nan, inf, 4., 5.], dtype=torch.float64)

In [24]:
print('removing invalid inputs..')
invalid_input_list = []
for i in range(len(arr)):
    x = arr[i]
    if torch.isinf(x).sum() > 0 or torch.isnan(x).sum() > 0:
        invalid_input_list.append(i)

valid_input_list = list(range(len(arr)))
for e in invalid_input_list:
    valid_input_list.remove(e)
train_features = arr[valid_input_list]

removing invalid inputs..


In [25]:
train_features

tensor([1., 2., 3., 4., 5.], dtype=torch.float64)

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
import tqdm

from dataset import Dataset
ds = Dataset(file_path='./data', window_size = 15, test_size = 50000)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.53it/s]


In [2]:
(train_features, train_labels), (test_features, test_labels) = ds.get_dataset()
print(train_features.shape, train_labels.shape)
print(test_features.shape, test_labels.shape)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:13<00:00, 12.31s/it]


['KRW-BCH', 'KRW-BTC', 'KRW-EOS', 'KRW-ETC', 'KRW-ETH', 'KRW-NEO'] are combined as training set
removing invalid inputs..
[]
completed
torch.Size([926504, 15, 17]) torch.Size([926504])
torch.Size([49985, 15, 17]) torch.Size([49985])
